# 📥 ETAPA 2: COLETA DE DOCUMENTOS DO GITLAB

## 🎯 **O que esta etapa faz**
Conecta ao repositório GitLab e baixa todos os documentos da pasta especificada para processamento local.

## 🤔 **Por que esta etapa é necessária**
Para processar documentos, primeiro precisamos:
- 📥 **Baixar os arquivos** do repositório GitLab
- 🔍 **Filtrar por tipo** (PDF, DOCX, MD, etc.)
- 📊 **Coletar metadados** (autor, data, tamanho)
- 💾 **Armazenar localmente** para processamento

## ⚙️ **Como funciona**
1. **Conecta ao GitLab** usando token de acesso
2. **Navega até a pasta** especificada (ex: 30-Aprovados)
3. **Lista todos os arquivos** na pasta
4. **Filtra por extensão** (.pdf, .docx, .md, .txt)
5. **Baixa cada arquivo** para a pasta local
6. **Coleta metadados** (autor, data de modificação, etc.)
7. **Gera relatório** de downloads realizados

## 📊 **Parâmetros que você pode ajustar**
- `target_folder`: Pasta no GitLab (padrão: "30-Aprovados")
- `file_extensions`: Tipos aceitos (padrão: [".pdf", ".docx", ".md", ".txt"])
- `max_file_size_mb`: Tamanho máximo (padrão: 50 MB)
- `branch`: Branch do repositório (padrão: "main")

## 👁️ **O que esperar de saída**
- 📁 **Pasta `documents/`** com todos os arquivos baixados
- 📄 **Arquivo JSON** com metadados de cada documento
- 📊 **Estatísticas** de download (sucessos, falhas, tamanhos)
- 📋 **Lista de próximos arquivos** para processamento

## ⚠️ **Pontos importantes**
- **Requer Etapa 1** (Fundação) concluída primeiro
- **Token GitLab** deve estar configurado
- **Conectividade** com GitLab é necessária
- **Modo independente** disponível para testes

## 🛡️ Verificação de Dependências

In [1]:
# 🛡️ VERIFICAÇÃO AUTOMÁTICA DE DEPENDÊNCIAS
import sys
import os
from pathlib import Path
from datetime import datetime

# Adicionar biblioteca ao path
sys.path.insert(0, str(Path().parent / "src"))

from pipeline_utils import (
    PipelineState, 
    check_prerequisites, 
    show_pipeline_progress
)

print("📥 ETAPA 2: COLETA DE DOCUMENTOS DO GITLAB")
print("=" * 60)
print(f"🕒 Iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verificar dependências
CURRENT_STAGE = 2
prerequisites_ok = check_prerequisites(CURRENT_STAGE)

if not prerequisites_ok:
    print("\n❌ ERRO: Dependências não atendidas!")
    print("📋 Ordem correta de execução:")
    for i in range(1, 8):
        status = "✅" if i < CURRENT_STAGE else "⏸️" if i == CURRENT_STAGE else "⏳"
        stage_names = ["", "🏗️ Fundação", "📥 GitLab", "⚙️ Docling", "🔪 Chunks", "🧠 Embeddings", "💾 Qdrant", "📊 Validação"]
        print(f"   {status} {i:02d}. {stage_names[i]}")
    
    print("\n🚨 EXECUTE PRIMEIRO: 01_FUNDACAO_PREPARACAO.ipynb")
    raise RuntimeError("Dependências não atendidas - execute etapas anteriores primeiro")

print("\n✅ Dependências verificadas - pode prosseguir")
print("📈 Progresso atual do pipeline:")
show_pipeline_progress()

📥 ETAPA 2: COLETA DE DOCUMENTOS DO GITLAB
🕒 Iniciado: 2025-08-17 18:39:02

✅ Dependências verificadas - pode prosseguir
📈 Progresso atual do pipeline:
🎯 PROGRESSO DO PIPELINE NIC ETL
════════════════════════════════════════════════════════════
✅ 01. 🏗️ Fundação e Preparação
⏳ 02. 📥 Coleta GitLab
⏳ 03. ⚙️ Processamento Docling
⏳ 04. 🔪 Segmentação em Chunks
⏳ 05. 🧠 Geração de Embeddings
⏳ 06. 💾 Armazenamento Qdrant
⏳ 07. 📊 Validação e Resultados
════════════════════════════════════════════════════════════


## 📋 Carregamento de Configuração

In [2]:
# 📋 CARREGAMENTO DA CONFIGURAÇÃO DA ETAPA ANTERIOR
print("📋 Carregando configuração da etapa anterior...")

# Carregar configuração real da etapa 1
state = PipelineState()
try:
    config = state.load_stage_data(1)
    print("✅ Configuração carregada da Etapa 1")
    print(f"   🔗 GitLab: {config['gitlab']['url']}")
    print(f"   📂 Repositório: {config['gitlab']['repository']}")
    print(f"   📁 Pasta alvo: {config['gitlab']['target_folder']}")
except Exception as e:
    print(f"❌ Erro ao carregar configuração: {e}")
    raise

print("\n" + "=" * 60)

📋 Carregando configuração da etapa anterior...
✅ Configuração carregada da Etapa 1
   🔗 GitLab: http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
   📂 Repositório: nic/documentacao/base-de-conhecimento
   📁 Pasta alvo: 30-Aprovados



## 📋 Configuração de Download

In [3]:
# 📋 CONFIGURAÇÃO DOS PARÂMETROS DE DOWNLOAD
print("📋 Configurando parâmetros de download...")

# 🎯 PARÂMETROS AJUSTÁVEIS
download_config = {
    # GitLab
    "gitlab_url": config["gitlab"]["url"],
    "repository": config["gitlab"]["repository"],
    "branch": config["gitlab"]["branch"],
    "target_folder": config["gitlab"]["target_folder"],
    "token": config["gitlab"]["token"],
    
    # Filtros
    "file_extensions": config["processing"]["file_extensions"],
    "max_file_size_mb": config["processing"]["max_file_size_mb"],
    
    # Destino
    "local_documents_dir": "../pipeline_data/documents",
    
    # Comportamento
    "overwrite_existing": True,
    "create_backup": False,
    "validate_downloads": True
}

print("\n📊 Parâmetros de download configurados:")
print(f"   🔗 URL: {download_config['gitlab_url']}")
print(f"   📂 Repositório: {download_config['repository']}")
print(f"   🌿 Branch: {download_config['branch']}")
print(f"   📁 Pasta remota: {download_config['target_folder']}")
print(f"   📁 Pasta local: {download_config['local_documents_dir']}")
print(f"   📄 Extensões: {', '.join(download_config['file_extensions'])}")
print(f"   📏 Tamanho máximo: {download_config['max_file_size_mb']} MB")
print(f"   🔄 Sobrescrever: {download_config['overwrite_existing']}")

# Verificar token
if download_config["token"] and download_config["token"] != "test_token_mock":
    token_preview = f"***{download_config['token'][-4:]}"
    print(f"   🔐 Token: {token_preview}")
else:
    print(f"   🔐 Token: {'🧪 Mock (teste)' if MODO_INDEPENDENTE else '❌ Não configurado'}")

print("\n✅ Configuração de download pronta")

📋 Configurando parâmetros de download...

📊 Parâmetros de download configurados:
   🔗 URL: http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
   📂 Repositório: nic/documentacao/base-de-conhecimento
   🌿 Branch: main
   📁 Pasta remota: 30-Aprovados
   📁 Pasta local: ../pipeline_data/documents
   📄 Extensões: .pdf, .docx, .md, .txt, .png, .jpg, .jpeg
   📏 Tamanho máximo: 50 MB
   🔄 Sobrescrever: True
   🔐 Token: ***DEN]

✅ Configuração de download pronta


## 🔗 Conexão e Validação do GitLab

In [4]:
# 🔗 CONEXÃO E VALIDAÇÃO DO GITLAB
print("🔗 Testando conexão com GitLab...")

from gitlab_utils import validate_gitlab_connection, list_repository_files

# Testar conexão
print("🔑 Validando credenciais...")
connection_valid = validate_gitlab_connection(
    gitlab_url=download_config["gitlab_url"],
    token=download_config["token"]
)

if not connection_valid:
    print("❌ Falha na conexão com GitLab")
    raise ConnectionError("Não foi possível conectar ao GitLab")

print("✅ Conexão estabelecida com sucesso")

# Listar arquivos na pasta alvo
print(f"\n📁 Listando arquivos na pasta: {download_config['target_folder']}")
files_list = list_repository_files(
    gitlab_url=download_config["gitlab_url"],
    repository=download_config["repository"],
    folder_path=download_config["target_folder"],
    token=download_config["token"],
    branch=download_config["branch"]
)

print(f"📄 Total de arquivos encontrados: {len(files_list)}")

# Filtrar por extensão e tamanho
print("🔍 Aplicando filtros...")
filtered_files = []

for file_info in files_list:
    # Verificar extensão
    has_valid_extension = any(
        file_info["name"].lower().endswith(ext.lower()) 
        for ext in download_config["file_extensions"]
    )
    
    # Verificar tamanho
    max_size_bytes = download_config["max_file_size_mb"] * 1024 * 1024
    is_valid_size = file_info["size"] <= max_size_bytes
    
    if has_valid_extension and is_valid_size:
        filtered_files.append(file_info)

print(f"📄 Arquivos após filtros: {len(filtered_files)}")

if filtered_files:
    print("\n📋 Arquivos que serão baixados:")
    total_size = 0
    for i, file_info in enumerate(filtered_files[:10], 1):  # Mostrar primeiros 10
        size_mb = file_info["size"] / (1024 * 1024)
        total_size += file_info["size"]
        print(f"   {i:2d}. {file_info['name']} ({size_mb:.1f} MB)")
    
    if len(filtered_files) > 10:
        remaining = len(filtered_files) - 10
        for file_info in filtered_files[10:]:
            total_size += file_info["size"]
        print(f"   ... e mais {remaining} arquivos")
    
    total_size_mb = total_size / (1024 * 1024)
    print(f"\n📏 Tamanho total: {total_size_mb:.1f} MB")
else:
    print("⚠️ Nenhum arquivo encontrado que atenda aos filtros")
    print("   📄 Extensões aceitas:", ", ".join(download_config["file_extensions"]))
    print(f"   📏 Tamanho máximo: {download_config['max_file_size_mb']} MB")

print("\n✅ Validação do GitLab concluída!")

🔗 Testando conexão com GitLab...
🔑 Validando credenciais...
🔍 Validando conexão com http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
🔗 URL original: http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git
🔗 URL da instância extraída: http://gitlab.processa.info
❌ Falha na validação: 401: 401 Unauthorized
❌ Falha na conexão com GitLab


ConnectionError: Não foi possível conectar ao GitLab

## 📥 Execução do Download dos Documentos

In [ ]:
# 📥 EXECUÇÃO DO DOWNLOAD DOS DOCUMENTOS
print("📥 Iniciando download dos documentos...")

from gitlab_utils import download_documents, clean_download_directory
from pipeline_utils import format_file_size
import json

# Preparar diretório de destino
documents_dir = Path(download_config["local_documents_dir"])
documents_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório se solicitado
if download_config["overwrite_existing"]:
    print("🧹 Limpando diretório de documentos...")
    clean_download_directory(str(documents_dir))

# Executar download
print(f"\n🚀 Iniciando download de documentos...")
print("=" * 50)

download_result = download_documents(
    gitlab_url=download_config["gitlab_url"],
    repository=download_config["repository"],
    branch=download_config["branch"],
    folder_path=download_config["target_folder"],
    token=download_config["token"],
    target_dir=str(documents_dir),
    file_extensions=download_config["file_extensions"]
)

print("\n" + "=" * 50)
print("📊 RESULTADO DO DOWNLOAD:")
print(f"   📄 Total encontrado: {download_result['statistics']['total_files_found']}")
print(f"   ✅ Baixados com sucesso: {download_result['statistics']['successfully_downloaded']}")
print(f"   ❌ Falhas: {download_result['statistics']['failed_downloads']}")
print(f"   📏 Tamanho total: {download_result['statistics']['total_size_mb']:.1f} MB")

# Mostrar arquivos baixados
if download_result["downloaded_files"]:
    print("\n📁 Arquivos baixados:")
    for file_info in download_result["downloaded_files"][:10]:  # Mostrar apenas os primeiros 10
        size_str = format_file_size(file_info["file_info"]["size_bytes"])
        print(f"   📄 {Path(file_info['local_path']).name} ({size_str})")
    
    if len(download_result["downloaded_files"]) > 10:
        remaining = len(download_result["downloaded_files"]) - 10
        print(f"   ... e mais {remaining} arquivos")

# Mostrar erros se houver
if download_result["errors"]:
    print(f"\n⚠️ Erros encontrados ({len(download_result['errors'])}):") 
    for error in download_result["errors"][:5]:  # Mostrar apenas os primeiros 5
        print(f"   ❌ {error}")
    
    if len(download_result["errors"]) > 5:
        remaining = len(download_result["errors"]) - 5
        print(f"   ... e mais {remaining} erros")

print("\n✅ Download concluído!")

## 📊 Validação e Estatísticas

In [ ]:
# 📊 VALIDAÇÃO DOS DOWNLOADS E ESTATÍSTICAS
print("📊 Validando downloads e coletando estatísticas...")

import os
from collections import defaultdict

# Verificar arquivos no diretório local
documents_dir = Path(download_config["local_documents_dir"])
local_files = list(documents_dir.glob("*"))
local_files = [f for f in local_files if f.is_file()]

print(f"\n📁 Arquivos no diretório local: {len(local_files)}")

# Estatísticas por tipo de arquivo
file_stats = defaultdict(lambda: {"count": 0, "total_size": 0})

total_size = 0
for file_path in local_files:
    file_size = file_path.stat().st_size
    file_ext = file_path.suffix.lower()
    
    file_stats[file_ext]["count"] += 1
    file_stats[file_ext]["total_size"] += file_size
    total_size += file_size

print("\n📈 Estatísticas por tipo de arquivo:")
for ext, stats in sorted(file_stats.items()):
    avg_size = stats["total_size"] / stats["count"] if stats["count"] > 0 else 0
    print(f"   {ext or 'sem ext'}: {stats['count']} arquivos, {format_file_size(stats['total_size'])} total, {format_file_size(avg_size)} médio")

print(f"\n📏 Tamanho total local: {format_file_size(total_size)}")

# Validar consistência
downloaded_count = download_result["statistics"]["successfully_downloaded"]
local_count = len(local_files)

if downloaded_count == local_count:
    print("✅ Consistência: Todos os downloads estão no diretório local")
else:
    print(f"⚠️ Inconsistência: {downloaded_count} baixados vs {local_count} locais")
    
    # Verificar quais arquivos estão faltando
    downloaded_names = {Path(f["local_path"]).name for f in download_result["downloaded_files"]}
    local_names = {f.name for f in local_files}
    
    missing = downloaded_names - local_names
    extra = local_names - downloaded_names
    
    if missing:
        print(f"   📄 Faltando: {', '.join(list(missing)[:5])}")
    if extra:
        print(f"   📄 Extras: {', '.join(list(extra)[:5])}")

# Preparar lista de arquivos para próxima etapa
next_stage_files = []
for file_path in local_files:
    # Encontrar metadados correspondentes
    file_metadata = None
    for downloaded_file in download_result["downloaded_files"]:
        if Path(downloaded_file["local_path"]).name == file_path.name:
            file_metadata = downloaded_file
            break
    
    file_info = {
        "local_path": str(file_path),
        "filename": file_path.name,
        "extension": file_path.suffix.lower(),
        "size_bytes": file_path.stat().st_size,
        "gitlab_metadata": file_metadata["file_info"].get("gitlab_metadata", {}) if file_metadata else {}
    }
    
    next_stage_files.append(file_info)

print(f"\n📋 Arquivos preparados para próxima etapa: {len(next_stage_files)}")
print("✅ Validação concluída!")

## 💾 Salvamento dos Resultados

In [ ]:
# 💾 SALVAMENTO DOS RESULTADOS PARA PRÓXIMA ETAPA
print("💾 Salvando resultados da coleta...")

from pipeline_utils import get_timestamp

# Preparar dados de saída para próxima etapa
stage_output = {
    "stage_info": {
        "stage_number": 2,
        "stage_name": "gitlab",
        "completed_at": get_timestamp(),
        "status": "success" if download_result["statistics"]["successfully_downloaded"] > 0 else "completed_with_warnings"
    },
    
    "download_config": {
        "gitlab_url": download_config["gitlab_url"],
        "repository": download_config["repository"],
        "branch": download_config["branch"],
        "target_folder": download_config["target_folder"],
        "file_extensions": download_config["file_extensions"],
        "max_file_size_mb": download_config["max_file_size_mb"]
    },
    
    "downloaded_files": download_result["downloaded_files"],
    
    "local_files": next_stage_files,
    
    "statistics": {
        "gitlab_files_found": download_result["statistics"]["total_files_found"],
        "successfully_downloaded": download_result["statistics"]["successfully_downloaded"],
        "failed_downloads": download_result["statistics"]["failed_downloads"],
        "local_files_count": len(next_stage_files),
        "total_size_mb": download_result["statistics"]["total_size_mb"],
        "file_types": dict(file_stats)
    },
    
    "errors": download_result["errors"],
    
    "next_stage_instructions": {
        "documents_directory": str(documents_dir),
        "files_to_process": len(next_stage_files),
        "processing_order": "by_size_ascending",
        "recommended_batch_size": min(config["pipeline"]["max_concurrent_docs"], len(next_stage_files))
    }
}

# Salvar usando PipelineState
state = PipelineState()
state.save_stage_data(2, stage_output)

# Marcar etapa como concluída
state.mark_stage_completed(2)

print(f"✅ Resultados salvos: {state.metadata_dir / 'stage_02_gitlab.json'}")
print(f"✅ Checkpoint criado: {state.checkpoints_dir / 'stage_02_completed.lock'}")

# Salvar log de download detalhado
download_log_path = documents_dir / "download_log.json"
with open(download_log_path, 'w', encoding='utf-8') as f:
    json.dump({
        "timestamp": get_timestamp(),
        "config": download_config,
        "results": download_result,
        "validation": {
            "local_files_count": len(local_files),
            "consistency_check": downloaded_count == local_count
        }
    }, f, indent=2, ensure_ascii=False, default=str)

print(f"📝 Log detalhado salvo: {download_log_path}")

# Exibir resumo final
print("\n" + "=" * 60)
print("🎉 ETAPA 2 CONCLUÍDA COM SUCESSO!")
print("=" * 60)

print(f"📊 Resumo da Coleta GitLab:")
print(f"   📂 Repositório: {download_config['repository']}")
print(f"   📁 Pasta: {download_config['target_folder']}")
print(f"   📄 Arquivos baixados: {download_result['statistics']['successfully_downloaded']}")
print(f"   📏 Tamanho total: {download_result['statistics']['total_size_mb']:.1f} MB")
print(f"   📁 Localização: {documents_dir}")

if download_result["errors"]:
    print(f"   ⚠️ Avisos: {len(download_result['errors'])} itens com problemas")
else:
    print(f"   ✅ Status: Todos os downloads bem-sucedidos")

print(f"\n🚀 Pronto para próxima etapa: 03_PROCESSAMENTO_DOCLING.ipynb")
print(f"📋 {len(next_stage_files)} documentos aguardando processamento")

# Exibir progresso do pipeline
print("\n📈 Progresso do Pipeline:")
show_pipeline_progress()

## 🔍 Informações de Debug (Opcional)

In [ ]:
# 🔍 INFORMAÇÕES DETALHADAS PARA DEBUG
# Execute esta célula se precisar de mais informações

def show_detailed_files():
    """Mostra lista detalhada de arquivos baixados"""
    
    print("📄 LISTA DETALHADA DE ARQUIVOS")
    print("=" * 50)
    
    for i, file_info in enumerate(next_stage_files, 1):
        size_str = format_file_size(file_info["size_bytes"])
        print(f"{i:2d}. {file_info['filename']}")
        print(f"     📏 Tamanho: {size_str}")
        print(f"     📄 Tipo: {file_info['extension']}")
        print(f"     📁 Local: {file_info['local_path']}")
        if file_info["gitlab_metadata"]:
            metadata = file_info["gitlab_metadata"]
            if "last_modified" in metadata:
                print(f"     📅 Modificado: {metadata['last_modified']}")
            if "author" in metadata:
                print(f"     👤 Autor: {metadata['author']}")
        print()

def show_download_statistics():
    """Mostra estatísticas detalhadas do download"""
    
    print("📊 ESTATÍSTICAS DETALHADAS")
    print("=" * 40)
    
    stats = download_result["statistics"]
    
    print(f"📡 GitLab:")
    print(f"   Arquivos encontrados: {stats['total_files_found']}")
    print(f"   Downloads bem-sucedidos: {stats['successfully_downloaded']}")
    print(f"   Downloads falharam: {stats['failed_downloads']}")
    
    if stats['successfully_downloaded'] > 0:
        success_rate = (stats['successfully_downloaded'] / stats['total_files_found']) * 100
        print(f"   Taxa de sucesso: {success_rate:.1f}%")
    
    print(f"\n💾 Armazenamento:")
    print(f"   Tamanho total: {stats['total_size_mb']:.2f} MB")
    
    if stats['successfully_downloaded'] > 0:
        avg_size = stats['total_size_mb'] / stats['successfully_downloaded']
        print(f"   Tamanho médio: {avg_size:.2f} MB por arquivo")
    
    print(f"\n📁 Por tipo de arquivo:")
    for ext, type_stats in file_stats.items():
        count = type_stats["count"]
        size_mb = type_stats["total_size"] / (1024 * 1024)
        print(f"   {ext or 'sem ext'}: {count} arquivos, {size_mb:.1f} MB")

def validate_file_integrity():
    """Valida integridade dos arquivos baixados"""
    
    print("🔍 VALIDAÇÃO DE INTEGRIDADE")
    print("=" * 35)
    
    corrupted_files = []
    
    for file_info in next_stage_files:
        file_path = Path(file_info["local_path"])
        
        # Verificar se arquivo existe
        if not file_path.exists():
            print(f"❌ Arquivo não encontrado: {file_path.name}")
            corrupted_files.append(file_path.name)
            continue
        
        # Verificar se arquivo não está vazio
        if file_path.stat().st_size == 0:
            print(f"⚠️ Arquivo vazio: {file_path.name}")
            corrupted_files.append(file_path.name)
            continue
        
        # Verificar se é legível (tentativa básica)
        try:
            with open(file_path, 'rb') as f:
                f.read(1024)  # Ler primeiro KB
            print(f"✅ {file_path.name}")
        except Exception as e:
            print(f"❌ Erro ao ler {file_path.name}: {e}")
            corrupted_files.append(file_path.name)
    
    if corrupted_files:
        print(f"\n⚠️ {len(corrupted_files)} arquivos com problemas")
    else:
        print(f"\n✅ Todos os {len(next_stage_files)} arquivos estão íntegros")

# Funções de debug disponíveis
print("🛠️ FUNÇÕES DE DEBUG DISPONÍVEIS:")
print("   show_detailed_files() - Lista detalhada de todos os arquivos")
print("   show_download_statistics() - Estatísticas completas do download")
print("   validate_file_integrity() - Verifica integridade dos arquivos")
print("\n💡 Execute as funções acima se precisar de mais informações")

---

## ✅ **Etapa 2 Concluída!**

### 🎯 **O que foi feito:**
- ✅ Conexão estabelecida com GitLab
- ✅ Documentos filtrados e baixados
- ✅ Metadados coletados e organizados
- ✅ Arquivos validados e catalogados
- ✅ Dados preparados para processamento

### 🚀 **Próxima etapa:**
**`03_PROCESSAMENTO_DOCLING.ipynb`** - Extração de conteúdo com Docling

### 📊 **Arquivos gerados:**
- `pipeline_data/documents/` - Documentos baixados do GitLab
- `pipeline_data/metadata/stage_02_gitlab.json` - Metadados e estatísticas
- `pipeline_data/documents/download_log.json` - Log detalhado do download
- `pipeline_data/checkpoints/stage_02_completed.lock` - Checkpoint de conclusão

### 📋 **Dados para próxima etapa:**
- **Documentos locais:** Prontos para extração de conteúdo
- **Metadados GitLab:** Autor, data de modificação, histórico
- **Estatísticas:** Tipos de arquivo, tamanhos, contagens

### 🔧 **Para executar próxima etapa:**
1. Abra o notebook `03_PROCESSAMENTO_DOCLING.ipynb`
2. Execute as células em sequência
3. Ou use o `00_PIPELINE_MASTER.ipynb` para execução automática

---

**📥 Documentos coletados com sucesso! Prontos para extração de conteúdo.**